In [ ]:
# !wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf


In [ ]:
# pip install git+https://github.com/abetlen/llama-cpp-python.git

In [ ]:
# pip install langchain-community

In [11]:
from langchain import LlamaCpp

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

llama_init_from_model: n_batch is less than GGML_KQ_MASK_PAD - increasing to 32
llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


In [12]:
llm.invoke("Hi! My name is Maarten. What is 1 + 1?")


''

In [13]:
from langchain import PromptTemplate

# Create a prompt template with the "input_prompt" variable
template = """<|user|>
{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(template=template, input_variables=["input_prompt"])

In [14]:
basic_chain = prompt | llm

In [15]:
basic_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


' Hello Maarten! The answer to 1 + 1 is 2.'

## Multiple Chains

In [24]:
from langchain import LLMChain

# Create a chain for the title of our story
template = """<|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""

title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

In [17]:
title.invoke({"summary": "a dog that lost its mother"})

{'summary': 'a dog that lost its mother',
 'title': ' "The Lone Pup: A Tale of Finding Strength After Mother\'s Absence"'}

In [18]:
# Create a chain for the character description using the summary and title
template = """<|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(template=template, input_variables=["summary", "title"])
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [19]:
# Create a chain for the story using the summary, title, and character description
template = """<|user|>
Create a story about {summary} with the title {title}. The main charachter is: {character}. Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title", "character"]
)
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [20]:
# Combine all three components to create the full chain
llm_chain = title | character | story

In [21]:
llm_chain.invoke("a dog that lost its mother")

{'summary': 'a dog that lost its mother',
 'title': ' The Lone Journey of Bella: Navigating Life Without Momma.',
 'character': " Bella, a spirited and resilient golden retriever, finds herself on an arduous journey after losing her beloved mother in a tragic accident. Her unwavering determination to survive and adapt fuels her lone journey as she navigates the challenges of life without Momma's warm embrace and guidance.\n\nThe main character, Bella, is portrayed with emotional depth and courage that captures both readers' hearts as they follow her solitary path toward growth and self-discovery in a world where companionship has been taken away from her by the loss of Momma.",
 'story': " In the heart of a bustling city, Bella, an intrepid golden retriever with sparkling amber eyes and a glossy coat that shone like sunlight on water, faced her lone journey after losing her beloved mother in a tragic accident. With unwavering determination and resilience etched into her spirit, she emb

## Memory

In [22]:
basic_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


' Hello Maarten! The answer to 1 + 1 is 2.'

In [23]:
# Next, we ask the LLM to reproduce the name
basic_chain.invoke({"input_prompt": "What is my name?"})

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


" I'm unable to determine your name as I don't have access to personal data of individuals. If you need assistance with something specific, feel free to ask!"

## ConversationBuffer

In [25]:
template = """<|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

In [26]:
from langchain.memory import ConversationBufferMemory

# Define the type of Memory we will use
memory = ConversationBufferMemory(memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-26-9823403f20ac>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [27]:
# Generate a conversation and ask a basic question
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})

{'input_prompt': 'Hi! My name is Maarten. What is 1 + 1?',
 'chat_history': '',
 'text': " The answer to 1 + 1 is 2. It's a basic arithmetic operation where you add one unit to another, resulting in two units total.\n\n---\n\nIf this were part of an ongoing conversation:\n\nHi Maarten! My name is [Assistant]. Regarding your question, the answer to 1 + 1 is indeed 2. It's always good to revisit fundamental math concepts like these—simple addition forms the foundation for more complex calculations later on. Anything else you'd like to know or discuss today?\n\nIf this were a standalone message:\n\nHello Maarten, I'm [Assistant]. The sum of 1 + 1 equals 2. If there are any other questions or topics you want to explore, feel free to ask!"}

In [28]:
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Maarten. What is 1 + 1?\nAI:  The answer to 1 + 1 is 2. It's a basic arithmetic operation where you add one unit to another, resulting in two units total.\n\n---\n\nIf this were part of an ongoing conversation:\n\nHi Maarten! My name is [Assistant]. Regarding your question, the answer to 1 + 1 is indeed 2. It's always good to revisit fundamental math concepts like these—simple addition forms the foundation for more complex calculations later on. Anything else you'd like to know or discuss today?\n\nIf this were a standalone message:\n\nHello Maarten, I'm [Assistant]. The sum of 1 + 1 equals 2. If there are any other questions or topics you want to explore, feel free to ask!",
 'text': " Hello Maarten, I'm [Assistant]. The sum of 1 + 1 equals 2. If there are any other questions or topics you want to explore, feel free to ask!\n\nYour name is Maarten as mentioned at the beginning of this conversation.\n\n-----"}

In [29]:
from langchain.memory import ConversationBufferWindowMemory

# Retain only the last 2 conversations in memory
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-29-046ef635f261>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")


In [30]:
llm_chain.invoke({"input_prompt":"Hi! My name is Maarten and I am 33 years old. What is 1 + 1?"})
llm_chain.invoke({"input_prompt":"What is 3 + 3?"})

{'input_prompt': 'What is 3 + 3?',
 'chat_history': "Human: Hi! My name is Maarten and I am 33 years old. What is 1 + 1?\nAI:  The answer to 1 + 1 is 2. While there's no need for extensive personal details in this context, I'm here to help with any questions you might have!",
 'text': ' The answer to 3 + 3 is 6. If you have any other math-related questions or need assistance, feel free to ask!'}

In [31]:
llm_chain.invoke({"input_prompt":"What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Maarten and I am 33 years old. What is 1 + 1?\nAI:  The answer to 1 + 1 is 2. While there's no need for extensive personal details in this context, I'm here to help with any questions you might have!\nHuman: What is 3 + 3?\nAI:  The answer to 3 + 3 is 6. If you have any other math-related questions or need assistance, feel free to ask!",
 'text': " Your name is Maarten.\n\nHowever, the information you provided about your name was in the initial conversation and not directly repeated here. If you're asking from this part of our current interaction, then I don't have that detail yet as it wasn't previously mentioned after my responses."}

In [32]:
llm_chain.invoke({"input_prompt":"What is my age?"})

{'input_prompt': 'What is my age?',
 'chat_history': "Human: What is 3 + 3?\nAI:  The answer to 3 + 3 is 6. If you have any other math-related questions or need assistance, feel free to ask!\nHuman: What is my name?\nAI:  Your name is Maarten.\n\nHowever, the information you provided about your name was in the initial conversation and not directly repeated here. If you're asking from this part of our current interaction, then I don't have that detail yet as it wasn't previously mentioned after my responses.",
 'text': " I'm an AI and do not have the ability to know personal information about individuals, including your age. This data is private, and you may choose to share it when you feel comfortable doing so in a secure manner. If you need assistance with calculating something or understanding concepts related to time, I can help explain those!"}

## ConversationSummary

In [33]:
summary_prompt_template = """<|user|>Summarize the conversations and update with the new lines.

Current summary:
{summary}

new lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template
)

In [34]:
from langchain.memory import ConversationSummaryMemory

# Define the type of memory we will use
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)

# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-34-905e44fca19a>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


In [35]:
# Generate a conversation and ask for the name
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': " Two cities that might experience an upsurge in diplomatic activities, taking into account the mentioned factors, are Istanbul, Turkey and Seoul, South Korea.\n\n\nIstanbul's geographical significance as a gateway between continents positions it favorably for negotiations related to the Middle East and European-Asian relations, despite current regional challenges. Its historical role in commerce and culture makes it an ideal venue for international diplomacy.\n\n\nSeoul is at the forefront of technological advancements, particularly in renewable energy, which aligns with global efforts to combat climate change. South Korea's vibrant economy and active cultural exchange programs also contribute to its potential as a nexus for diplomatic engagement, especially considering its alliances and economic partnerships that could be bolstered by dialogue on sustainable technologies.",
 'text': " Your name is not explicitly provided in the cu

In [36]:
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

{'input_prompt': 'What was the first question I asked?',
 'chat_history': " Considering the discussed factors, Istanbul and Seoul are identified as two cities with significant potential for increased diplomatic activities. Istanbul's strategic location between Europe and Asia makes it a key player in Middle Eastern and European-Asian relations, while its rich history of trade and culture provides a strong foundation for international diplomacy despite regional challenges. On the other hand, Seoul stands out as a leader in technological innovation, particularly renewable energy solutions, which align with global sustainability efforts. South Korea's robust economy and active cultural exchange initiatives, coupled with its strategic alliances, further underscore its suitability as a hub for diplomatic discussions focusing on green technology and economic partnerships.",
 'text': ' It appears there was no direct question preceding the detailed explanation about Istanbul and Seoul\'s poten

In [37]:
memory.load_memory_variables({})

{'chat_history': " The conversation revolves around identifying cities with significant potential for enhanced diplomatic activities, focusing on Istanbul and Seoul. Istanbul's strategic location between Europe and Asia positions it as a key player in Middle Eastern and European-Asian relations, offering opportunities for international diplomacy despite regional challenges. Its rich history of trade and culture strengthens its suitability for global diplomatic engagements. Conversely, Seoul is recognized for its leadership in technological innovation, especially renewable energy solutions, aligning with global sustainability goals. South Korea's strong economy, cultural exchanges, and strategic alliances make it an ideal hub for discussions on green technology and economic partnerships."}

## Agents

In [ ]:
# pip install langchain-openai

In [ ]:
# pip install -U duckduckgo-search

In [54]:
import os
from langchain_openai import ChatOpenAI

# Load OpenAI's LLMs with LangChain
os.environ["OPENAI_API_KEY"] = "my_key"
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [50]:
# Create the ReAct template
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [51]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# Prepare tools
tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [52]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent
agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [53]:
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)



> Entering new AgentExecutor chain...
I need to find the current price of a MacBook Pro in USD and then convert it to EUR using the given exchange rate.
Action: duckduck
Action Input: "current price of MacBook Pro in USD"snippet: A MacBook Pro cost ranges from just over $1000 to well over $3000. Determining the true cost for the MacBook Pro you need requires close examination of the model options, hardware configurations, and Apple's pricing strategies. Let's examine how Apple prices each MacBook Pro base model and default hardware configuration: MacBook Pro 13-inch, title: How Much Does a MacBook Pro Cost? - The Pricer, link: https://www.thepricer.org/how-much-does-a-macbook-pro-cost/, snippet: If you want the latest MacBook Pro with a larger display, you'll also have to upgrade to the 14-core M4 Pro chip. ... Best price (current) Best price (all-time) M2 MacBook Air (13-inch) $999: $799 ..., title: Best MacBook Deals: Save Big on the Latest M4 Laptops. Plus ... - CNET, link: https:

{'input': 'What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?',
 'output': 'The current price of a MacBook Pro in USD is $2499. If the exchange rate is 0.85 EUR for 1 USD, it would cost approximately 2124.15 EUR.'}